In [1]:
from langchain_community.tools import WikipediaQueryRun  ## execute a query and retrieve the results
from langchain_community.utilities import WikipediaAPIWrapper   ## set up the connection or configuration for interacting with Wikipedia.

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\AHMED ABD ELGWAD\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=500))

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load documents from the web
load_web = WebBaseLoader("https://docs.smith.langchain.com/")
docs_web = load_web.load()

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents into chunks
split_web = text_splitter.split_documents(docs_web)

# Initialize the embedding model
embedding_model = OllamaEmbeddings(model="llama3.2:3b")

# Initialize the vector database
vector_db = FAISS.from_documents(split_web, embedding_model)

# Initialize the retriever
retriever_web = vector_db.as_retriever()

retriever_web

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\AHMED ABD ELGWAD\AppData\Local\Temp\ipykernel_10088\2330395285.py:17: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="llama3.2:3b")


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024134970A40>, search_kwargs={})

In [5]:
# Create a retriever tool for the vector database
from langchain.tools.retriever import create_retriever_tool         ## used to search for information in a vector database or other data source

# Create the retriever tool with the vector database and a description
retriever_web_tool = create_retriever_tool(retriever_web,
                    "langsmith_search",  ## used when integrating the tool into an agent or pipeline 
                    "Search for information about LangSmith.For any questions about LangSmith, you must use this tool!")

retriever_web_tool.name

'langsmith_search'

In [6]:
# Arxiv Tool and Utilities for Creating and Reading Applications
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun   ##  query the Arxiv database and retrieve relevant information

# Set up the connection to Arxiv API
arxiv_api = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)

# Create an ArxivQueryRun tool
arxiv_query_run = ArxivQueryRun(arxiv_api=arxiv_api)

arxiv_query_run.name

'arxiv'

In [ ]:
# Define the tools list
tools = [wiki, retriever_web_tool, arxiv_query_run]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\AHMED ABD ELGWAD\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=500)),
 Tool(name='langsmith_search', description='Search for information about LangSmith.For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000024124E46480>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024134970A40>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000024

In [ ]:
# Initialize the Ollama LLM
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

llm

C:\Users\AHMED ABD ELGWAD\AppData\Local\Temp\ipykernel_10088\1732197747.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


Ollama()

In [9]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

prompt.messages

c:\Users\AHMED ABD ELGWAD\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [10]:
## agent
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [11]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [ ]:
response = agent_executor.invoke({"input": "Tell me about LangSmith"})
response

In [30]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings  # Use local embeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain_community.chat_models import ChatOllama  # Use Ollama instead of OpenAI

# Set up Wikipedia search
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

# Load and process LangSmith documentation
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents into chunks
split_web = text_splitter.split_documents(docs_web)

# Initialize the embedding model
embedding_model = OllamaEmbeddings(model="llama3.2:3b")

# Initialize the vector database
vector_db = FAISS.from_documents(split_web, embedding_model)

# Initialize the retriever
retriever_web = vector_db.as_retriever()

retriever_tool = create_retriever_tool(
    retriever_web,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)

# Set up Arxiv search
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

# Define tools
tools = [wiki, arxiv, retriever_tool]

# Replace OpenAI with Ollama
llm = ChatOllama(model="llama2")  # Change to your desired Ollama model

# Get the prompt template
prompt = hub.pull("hwchase17/openai-functions-agent")

# Create the agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
response = agent_executor.invoke({"input": "Tell me about Langsmith"})
print(response)

c:\Users\AHMED ABD ELGWAD\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

Langsmith is a small town located in the heart of the English countryside. It is known for its picturesque landscape, rich history, and friendly locals. Here are some interesting facts about Langsmith:

1. Language Smith: The town's name has a fascinating origin story. According to local legend, the town was named after a medieval smith (blacksmith) named Langley who lived in the area. Over time, the name evolved into "Langsmith."
2. Historic Churches: Langsmith is home to several beautiful churches, including St. Nicholas' Church, which dates back to the 13th century. The church features intricate stone carvings and stunning stained-glass windows.
3. Agricultural Heritage: Langsmith has a long history of agriculture, with many farms and fields surrounding the town. The area is known for its fertile soil and mild climate, making it an ideal place for growing crops like wheat, barley, and vegetables.
4. Picturesque Villages: Langsmith is surround